# *IV Curves*

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%autosave 0

In [ ]:
from pathlib import Path
from measurements_table import create_measurements_table
import utils
import pandas
import plotly.express as px

# Read data

In [ ]:
IV_measurements_table_df = create_measurements_table().query('Type=="IV curve"')

measured_data_df = pandas.DataFrame()
for measurement_name in IV_measurements_table_df.index:
    df = pandas.read_feather(utils.path_to_measurements_directory/Path(measurement_name)/Path('IV_curve/measured_data.fd'))
    df['Bias voltage (V)'] *= -1
    df['Bias current (A)'] *= -1
    df['Measurement name'] = measurement_name
    device_name = IV_measurements_table_df.loc[measurement_name, 'Measured device']
    df['Device'] = device_name
    measured_data_df = measured_data_df.append(df, ignore_index=True)
display(measured_data_df)

Add some rows with information of each device:

In [ ]:
measured_data_df.set_index('Device', inplace=True)
measured_data_df = measured_data_df.join(utils.bureaucrat.devices_sheet_df)
measured_data_df.reset_index(inplace=True)
display(measured_data_df)
print(sorted(measured_data_df.columns))

Calculate mean and std at each `n_voltage`:

In [ ]:
mean_std_df = measured_data_df.copy()
mean_std_df.drop(columns=['was measured before irradiation','When','x','y','status','seems dead?','irradiate with','fluence','comments','color'], inplace=True)
mean_std_df = utils.mean_std(
    mean_std_df,
    by = list(set(mean_std_df.columns).difference({
        'Bias current (A)',
        'Bias voltage (V)',
        'n_trigger',
    }))
)
display(mean_std_df)
print(sorted(mean_std_df.columns))

# Plots

In [ ]:
df = mean_std_df.copy()
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = df.sort_values(['Device','n_voltage']).reset_index(),
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    error_y = 'Bias current (A) std',
    error_y_mode = 'band',
    facet_col = 'wafer',
    facet_row = 'trenches',
    color = 'trench depth',
    symbol = 'pixel border',
    line_dash = 'contact type',
    grouped_legend = True,
    hover_name = 'Measurement name',
    #text = 'Device',
    log_y = True,
    line_group = 'Measurement name',
)
fig.show()

## Plots for the RD50 Workshop

In [ ]:
df = mean_std_df.reset_index()
df = df.loc[df['Device'].isin({ # Keep only the devices that I am showing in the presentation.
    '4','7','14','20','23','34','36','45','52','54','65','68','78','84','87',
    '16','50','81', # These are those devices that were never alive (I suspect due to some design bug or so).
})]
df = df.loc[~df['Measurement name'].isin({ # Remove some specific measurements.
    '20211024131707_#65_IV_curve', # Not all pads grounded.
    '20211023104738_#45_IVCurve', # This is one of the first measurements I did. Later on I re-measured #45 and it was much better, don't know what happened in this particular measurement.
    '20211108170237_#20_after_going_to_PSI', # Duplicated for this device.
    '20211108171401_#78_after_going_to_PSI', # Duplicated for this device.
})]
df = df.loc[df['Bias voltage (V) mean'].diff()>0] # Find rows in which voltage is increasing to remove the "dual sweep".
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = df.sort_values(['trenches','trench depth','Device','n_voltage']),
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    error_y = 'Bias current (A) std',
    error_y_mode = 'band',
    facet_col = 'wafer',
    facet_row = 'trenches',
    color = 'trench depth',
    symbol = 'pixel border',
    line_dash = 'contact type',
    grouped_legend = True,
    hover_name = 'Measurement name',
   # text = 'Device',
    log_y = True,
    line_group = 'Measurement name',
)
fig.show()
fig.write_html(f'iv_curves.html', include_plotlyjs='cdn')

# The death of device #2

In [ ]:
fig = utils.line(
    title = 'TI-LGADs IV curves',
    data_frame = mean_std_df.query('Device=="2"').sort_values(['Device','n_voltage']).reset_index(),
    x = 'Bias voltage (V) mean',
    y = 'Bias current (A) mean',
    labels = {
        'Bias voltage (V) mean': 'Bias voltage (V)',
        'Bias current (A) mean': 'Bias current (A)',
    },
    symbol = 'Device',
    hover_name = 'Measurement name',
    log_y = True,
    line_group = 'Measurement name',
    animation_frame = 'Measurement name'
)
fig.show()